In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType


In [ ]:
# Reloadable libraries
import finite_diff, importlib
importlib.reload(finite_diff)
from finite_diff import fd_hessian_test, fd_gradient_test, gradient_convergence_plot, hessian_convergence_plot, hessian_convergence_block_plot

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/sphere.obj')
SURFACE_PATH = osp.join(weaving_dir + 'surface_models/sphere.obj')
SUBDIVISION_RESOLUTION = 5

In [ ]:
linkage = elastic_rods.SurfaceAttractedLinkage(SURFACE_PATH, True, MODEL_PATH, SUBDIVISION_RESOLUTION, False)
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
linkage.updateSourceFrame()
linkage.set_design_parameter_config(True, True)

In [ ]:
linkage.attraction_weight = 1000

In [ ]:
direction = np.random.normal(-1, 1, linkage.numDoF())

In [ ]:
grad = linkage.gradientPerSegmentRestlen()

In [ ]:
energyType = elastic_rods.SurfaceAttractionEnergyType.Attraction

In [ ]:
gradient_convergence_plot(linkage, perSegmentRestLen=True, etype=energyType)

In [ ]:
hessian_convergence_plot(linkage,  minStepSize=1e-12, maxStepSize=1e5, etype = elastic_rods.SurfaceAttractionEnergyType.Attraction, direction = direction)

In [ ]:
jv_offset = linkage.dofOffsetForJoint(0)
rk_offset = linkage.numDoF()
rl_offset = rk_offset + (linkage.numRestKappaVars() if linkage.get_design_parameter_config().restKappa else 0)

In [ ]:
var_types = ['FULL', 'RV', 'JV', 'RK', 'PSRL']
var_indices = {'FULL': range(0, linkage.numExtendedDoFPSRL()),
                 'RV': range(0, jv_offset),
                'JV' : range(jv_offset, rk_offset),
                 'RK': range(rk_offset, rl_offset),
                 'PSRL': range(rl_offset, linkage.numExtendedDoFPSRL())
              }

In [ ]:
hessian_convergence_block_plot(linkage, var_types, var_indices, variableDesignParameters=False, perSegmentRestLen=True, etype=elastic_rods.SurfaceAttractionEnergyType.Full)